In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

windows = True # If on mac put false, windows true

First section is checking that it reconfirms baselines values  

Second section will be running for the end flow values. This will be taken from a new function which stores the values in _storage and we will import

In [6]:
if windows == True:
    baselines = pd.read_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\baselines_met.csv", index_col=0).rename(columns={"Q in single(µm3/s)": "baseline Q"}) # FOR WINDOWS
else:
    baselines = pd.read_csv(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/baselines_met.csv", index_col=0).rename(columns={"Q in single(µm3/s)": "baseline Q"}) # FOR MAC

In [7]:
H = 0.42 #no units,ratio #Hematocrit assumed to be constant
# M_o = 30* 10e-9 #mol_ O2/(mL/s) #Taken from table 2 from Wiley Payne paper
cHb = 0.2 #mL_O2/m #Taken from table 2 from Wiley Payne paper
K_o = 5e-8 #µL/(mm*s*mmHg) #payne paper and boas et al

#convert param to my SU
M = 8.20e-4 # cm3_O2/(cm3*s) , still unsure about the exact conversion so will just input in this section
K = K_o * (10**6) # µm2/(s*mmHg)

In [8]:
# display(baselines)
df = baselines.drop(['Saturation in','Pressure Drop(mmHg)','Saturation out','Saturation ave','partial pressure blood(mmHg)','tissue partials(mmHg)','dS'],axis=1)
df.at[0,'Saturation in'] = 0.94

In [9]:
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 10e-13 and all_roots[i].imag >= -10e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols

In [10]:
df['pb-pt'] = M*df['wall thickness(µm)']*df['Vt(µm3)']/(2*np.pi*K*0.5*df['Diameter(µm)']*df['Length(µm)'])

for i in range(len(df)):
    if i != 0:
        df.at[i,'Saturation in'] = df.at[i-1,'Saturation out']
        
    df.at[i,'Saturation out'] = df.at[i,'Saturation in'] - ( (2*np.pi*K*0.5*df.at[i,'Diameter(µm)']*df.at[i,'Length(µm)']) / (df.at[i,'wall thickness(µm)']*cHb*H) ) * df.at[i,'pb-pt'] / df.at[i,'baseline Q']

df['Saturation ave'] = ( df['Saturation in'] + df['Saturation out'] ) / 2 

for j in range(len(df['Saturation ave'])):
        root_temp = None
        sols = 0
        root_temp = np.roots([1,0,150,23400*df.loc[j,'Saturation ave']/(df.loc[j,'Saturation ave']-1)]) #gives mmHg , *133.322
        sols = check_imag_roots_real(root_temp) 
        #sols = sols*133.322 ###Remove this step if want in mmHg
        df.loc[j,'partial pressure blood(mmHg)'] = sols

df['partial pressure tissue(mmHg)'] = df['partial pressure blood(mmHg)'] - df['pb-pt']

# display(df)
# display(baselines)

# For final flow values

In [11]:
#ss stands for steady state
if windows == True:
    ss = pd.read_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\final_nometp_7_62.csv", index_col=0) # FOR WINDOWS
else:
    ss = pd.read_csv(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/final.csv", index_col=0) # FOR MAC

In [12]:
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 10e-13 and all_roots[i].imag >= -10e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols

In [13]:
ss['pb-pt'] = M*ss['wall thickness(µm)']*ss['Vt(µm3)']/(2*np.pi*K*0.5*ss['Diameter(µm)']*ss['Length(µm)'])

for i in range(len(ss)):
    if i != 0:
        ss.at[i,'Saturation in'] = ss.at[i-1,'Saturation out']
        
    ss.at[i,'Saturation out'] = ss.at[i,'Saturation in'] - ( (2*np.pi*K*0.5*ss.at[i,'Diameter(µm)']*ss.at[i,'Length(µm)']) / (ss.at[i,'wall thickness(µm)']*cHb*H) ) * ss.at[i,'pb-pt'] / ss.at[i,'Q(µm3/s)']

ss['Saturation ave'] = ( ss['Saturation in'] + ss['Saturation out'] ) / 2 

for j in range(len(ss['Saturation ave'])):
        root_temp = None
        sols = 0
        root_temp = np.roots([1,0,150,23400*ss.loc[j,'Saturation ave']/(ss.loc[j,'Saturation ave']-1)]) #gives mmHg , *133.322
        sols = check_imag_roots_real(root_temp) 
        #sols = sols*133.322 ###Remove this step if want in mmHg
        ss.loc[j,'partial pressure blood(mmHg)'] = sols

ss['partial pressure tissue(mmHg)'] = ss['partial pressure blood(mmHg)'] - ss['pb-pt']

# display(ss)

In [14]:
view = pd.DataFrame()
view['Name'] = ss['Name']
view['final Saturation in'] = ss['Saturation in']
view['-change from baseline Sat in'] = baselines.loc[:,'Saturation in']-ss.loc[:,'Saturation in']
view['final Saturation out'] = ss['Saturation out']
view['-change from baseline Sat out'] = baselines.loc[:,'Saturation out']-ss.loc[:,'Saturation out']
view['final pb(mmHg)'] = ss['partial pressure blood(mmHg)']
view['-change from baseline pb(mmHg)'] = baselines.loc[:,'partial pressure blood(mmHg)']-ss.loc[:,'partial pressure blood(mmHg)']
view['final pt(mmHG)'] = ss['partial pressure tissue(mmHg)']
view['-change from baseline pt(mmHg)'] = baselines.loc[:,'tissue partials(mmHg)']-ss.loc[:,'partial pressure tissue(mmHg)']

# view[''] = baselines.loc[:,'']-ss.loc[:,'']

display(view)

# To find out information on what values were used and for how long this was run for etc. 
# just go look at the 1_finak_flows notebook.

,Name,final Saturation in,-change from baseline Sat in,final Saturation out,-change from baseline Sat out,final pb(mmHg),-change from baseline pb(mmHg),final pt(mmHG),-change from baseline pt(mmHg)
0,A1,0.940000,0.000000,0.920173,0.009827,67.035624,1.807507,63.774974,1.903863
1,A2,0.920173,0.009827,0.900301,0.019699,61.101896,4.225902,58.655983,4.301211
2,A3,0.900301,0.019699,0.840371,0.049629,53.027921,6.781141,45.933822,7.039235
3,A4,0.840371,0.049629,0.739358,0.100642,43.348268,8.835712,30.908545,9.423434
4,A5,0.739358,0.100642,0.584020,0.180980,34.372835,10.173891,12.912856,11.781707
5,A6,0.584020,0.180980,0.372807,0.292193,25.994086,11.584927,7.114967,13.271576
6,C,0.372807,0.292193,0.259954,0.350046,19.873031,13.204269,16.914319,13.387461
7,V6,0.259954,0.350046,0.234423,0.363077,17.214476,14.180530,16.696121,14.212034
8,V5,0.234423,0.363077,0.213776,0.373724,16.281768,14.579932,15.025972,14.659199
9,V4,0.213776,0.373724,0.203560,0.378940,15.638526,14.874363,14.927923,14.918989
